In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# To fix transactions to add stores with no previous values as 0 values
import janitor

In [69]:
test = pd.read_csv('test.csv', parse_dates=['date'])
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv', parse_dates=['date'])
holidays = pd.read_csv('holidays_events.csv', parse_dates=['date'])
oil = pd.read_csv('oil.csv', parse_dates=['date'])
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv', parse_dates=['date'])

To set up date ranges for the train and test dataframes. 

In [3]:
train_start = train.date.min().date()
train_end = train.date.max().date()
num_train_date = train.date.nunique()
train_len = (train_end - train_start).days + 1
test_start = test.date.min().date()
test_end = test.date.max().date()
num_test_date = test.date.nunique()

In [4]:
test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [5]:
test.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
onpromotion             int64
dtype: object

In [6]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [7]:
train.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
dtype: object

In [8]:
train['family'].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [9]:
test['family'].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [10]:
transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


To use the janitor function to add 0 values for transactions for stores that don't have any transactions listed on a particular date. This will make it possible to merge transactions into train_test dataframe.

In [11]:
transactions = transactions.complete('date', 'store_nbr').fillna(0, downcast='infer')

In [12]:
transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-01,1,0
2,2013-01-01,2,0
3,2013-01-01,3,0
4,2013-01-01,4,0
...,...,...,...
90823,2017-08-15,29,1302
90824,2017-08-15,21,1059
90825,2017-08-15,42,995
90826,2017-08-15,22,766


In [13]:
sample

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [70]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


To look at the missing oil dates. And it seems the number of those missing dates match the number of weekends in the the overall time period. 

In [71]:
missing_oil_dates = pd.date_range(train_start, test_end).difference(oil.date)
num_missing_oil_dates = len(missing_oil_dates)
print(num_missing_oil_dates)

486


In [72]:
num_wknd_missing = (missing_oil_dates.weekday >= 5).sum()
print(num_wknd_missing)

486


In [73]:
total_num_wknd = (pd.date_range(train_start, test_end).weekday >= 5).sum()
print(total_num_wknd)

486


Because grocery stores open on weekends as well, there is a need to estimate the missing oil prices if they are to be used in the models. For simplicity, the missing values in the oil column are filled using linear interpolation, where the estimated value is assumed to lie on the line joining the nearest values to the left and right.

In [74]:
oil = oil.merge(
    pd.DataFrame({"date": pd.date_range(train_start, test_end)}),
    on="date",
    how="outer",
).sort_values("date", ignore_index=True)

oil = oil.set_index('date').dcoilwtico.interpolate(method='linear').to_frame()

In [77]:
oil

,dcoilwtico
date,
2013-01-01,NaN
2013-01-02,93.140000
2013-01-03,92.970000
2013-01-04,93.120000
2013-01-05,93.146667
...,...
2017-08-27,46.816667
2017-08-28,46.400000
2017-08-29,46.460000


In [20]:
holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [24]:
stores

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [25]:
pd.isnull(test).sum()[pd.isnull(test).sum() > 0]

Series([], dtype: int64)

In [26]:
pd.isnull(train).sum()[pd.isnull(train).sum() > 0]

Series([], dtype: int64)

In [27]:
pd.isnull(transactions).sum()[pd.isnull(transactions).sum() > 0]

Series([], dtype: int64)

In [78]:
pd.isnull(oil).sum()[pd.isnull(oil).sum() > 0]

dcoilwtico    1
dtype: int64

In [29]:
pd.isnull(holidays).sum()[pd.isnull(holidays).sum() > 0]

Series([], dtype: int64)

In [30]:
pd.isnull(stores).sum()[pd.isnull(stores).sum() > 0]

Series([], dtype: int64)

To change the oil NaN to 0.

In [79]:
oil = oil.fillna(0)

In [80]:
pd.isnull(oil).sum()[pd.isnull(oil).sum() > 0]

Series([], dtype: int64)

To merge train and test just for initial data manipulation.

In [59]:
train_test = pd.concat([train, test], ignore_index=True)

In [60]:
train_test

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0
...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9


In [61]:
train_test.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
dtype: object

To create a day of the week column for the dataframe and convert to numbers. 

In [62]:
train_test['day_of_week'] = train_test['date'].dt.day_name()
mapping = {
    'Sunday': 7,
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday' : 4,
    'Friday' : 5,
    'Saturday' : 6
}
train_test['day_of_week'] = train_test['day_of_week'].replace(mapping)

To create a weekend column to denote whether the day of the week is a weekend or not. 

In [63]:
train_test['weekend'] = pd.Series(0, index=train_test.index).mask(train_test['day_of_week']>5, 1)

In [64]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0
4,4,2013-01-01,1,BOOKS,0.0,0,2,0
...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4,0
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4,0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4,0
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4,0


To create a year, month and day of month columns.

In [65]:
train_test['year'] = train_test.date.dt.year
train_test['month'] = train_test.date.dt.month
train_test['day_of_month'] = train_test.date.dt.day

In [66]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,year,month,day_of_month
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,2013,1,1
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,2013,1,1
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,2013,1,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,2013,1,1
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,2013,1,1
...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4,0,2017,8,31
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4,0,2017,8,31
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4,0,2017,8,31
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4,0,2017,8,31


To merge the oil values into the train_test dataframe. For some reason it added 3 additional rows on the bottom, so these were eliminated by looking for those rows where id doesn't exist. Also, for some reason it created NaNs for dates where there were no oil production, so those had to be eliminated. 

In [81]:
train_test = pd.merge(train_test, oil, on='date', how='outer')

In [82]:
train_test = train_test[train_test['id'].notna()]
train_test['dcoilwtico'].fillna(0)

0           0.00
1           0.00
2           0.00
3           0.00
4           0.00
           ...  
3029395    47.26
3029396    47.26
3029397    47.26
3029398    47.26
3029399    47.26
Name: dcoilwtico, Length: 3029400, dtype: float64

In [298]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico
0,0.0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
1,1.0,2013-01-01,1.0,BABY CARE,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
2,2.0,2013-01-01,1.0,BEAUTY,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
3,3.0,2013-01-01,1.0,BEVERAGES,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
4,4.0,2013-01-01,1.0,BOOKS,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395.0,2017-08-31,9.0,POULTRY,NaN,1.0,4.0,0.0,8.0,243.0,35,2017.0,47.26
3029396,3029396.0,2017-08-31,9.0,PREPARED FOODS,NaN,0.0,4.0,0.0,8.0,243.0,35,2017.0,47.26
3029397,3029397.0,2017-08-31,9.0,PRODUCE,NaN,1.0,4.0,0.0,8.0,243.0,35,2017.0,47.26
3029398,3029398.0,2017-08-31,9.0,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,4.0,0.0,8.0,243.0,35,2017.0,47.26


For some reason this outer join converted the integer values to floating values, so these needed to be reinstated. 

In [83]:
train_test['id']=train_test['id'].astype(np.int64)
train_test['store_nbr']=train_test['store_nbr'].astype(np.int64)
train_test['onpromotion']=train_test['onpromotion'].astype(np.int64)

In [84]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,year,month,day_of_month,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00
1,1,2013-01-01,1,BABY CARE,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00
2,2,2013-01-01,1,BEAUTY,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00
3,3,2013-01-01,1,BEVERAGES,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00
4,4,2013-01-01,1,BOOKS,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4.0,0.0,2017.0,8.0,31.0,47.26
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4.0,0.0,2017.0,8.0,31.0,47.26
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4.0,0.0,2017.0,8.0,31.0,47.26
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4.0,0.0,2017.0,8.0,31.0,47.26


To merge the stores dataframe into the train_test dataframe.

In [85]:
train_test = pd.merge(train_test, stores, on='store_nbr', how='outer')

In [86]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,year,month,day_of_month,dcoilwtico,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,2.0,0.0,2013.0,1.0,1.0,0.00,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4.0,0.0,2017.0,8.0,31.0,47.26,Quito,Pichincha,B,6
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4.0,0.0,2017.0,8.0,31.0,47.26,Quito,Pichincha,B,6
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4.0,0.0,2017.0,8.0,31.0,47.26,Quito,Pichincha,B,6
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4.0,0.0,2017.0,8.0,31.0,47.26,Quito,Pichincha,B,6


To merge the transactions dataframe into the train_test dataframe on both date and store number since the same transactions need to be show by store by date for all the different product sales for each day and store. 

In [303]:
#train_test = pd.merge(train_test, transactions, on=['date', 'store_nbr'], how='outer')

In [304]:
#train_test['transactions'].fillna(0, inplace=True)

In [305]:
#train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0


In [103]:
holidays = pd.read_csv('holidays_events.csv')

holidays.query('transferred==True')

# transferred day is not celebrated
holidays = holidays.query('transferred ==False')
holidays.description = holidays.description.str.replace('Traslado ','')

#national

national = holidays.query('locale=="National"')

day_off = national.query('type!="Work Day" or type!="Event"').set_index('date')['description'].to_dict()

train_test['date_str'] = train_test.date.astype(str)

train_test['national_holiday'] = [1 if a in day_off else 0 for a in train_test.date_str]

event = national.query('type=="Event"').set_index('date')['description'].to_dict()

train_test['national_event'] =[1 if a in event else 0 for a in train_test.date_str]

work_day = national.query('type=="Work Day"').set_index('date')['description'].to_dict()
train_test['national_workday'] = [1 if a in work_day else 0 for a in train_test.date_str]

In [104]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,year,month,...,city,state,type,cluster,date_str,national_holiday,national_event,national_workday,local_holiday,regional_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0


In [106]:
local = holidays.query('locale=="Local"')
local_dic = local.set_index('date').locale_name.to_dict()
train_test['local_holiday']=[1 if b in local_dic and local_dic[b]== a else 0 for a,b in zip(train_test.city,train_test.date_str)]

In [107]:
regional = holidays.query('locale=="Regional"')
regional_dic = regional.set_index('date').locale_name.to_dict()
train_test['regional_holiday']= [1 if b in regional_dic and regional_dic[b]== a else 0 for a,b in zip(train_test.state,train_test.date_str)]

In [110]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,year,month,...,city,state,type,cluster,date_str,national_holiday,national_event,national_workday,local_holiday,regional_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0


In [111]:
train_test.dtypes

id                           int64
date                datetime64[ns]
store_nbr                    int64
family                      object
sales                      float64
onpromotion                  int64
day_of_week                  int64
weekend                      int64
year                         int64
month                        int64
day_of_month                 int64
dcoilwtico                 float64
city                        object
state                       object
type                        object
cluster                      int64
date_str                    object
national_holiday             int64
national_event               int64
national_workday             int64
local_holiday                int64
regional_holiday             int64
dtype: object

To clean up the formatting of the float variables. 

In [112]:
pd.options.mode.chained_assignment = None 

In [113]:
train_test['day_of_week']=train_test['day_of_week'].astype(np.int64)
train_test['weekend']=train_test['weekend'].astype(np.int64)
train_test['month']=train_test['month'].astype(np.int64)
train_test['day_of_month']=train_test['day_of_month'].astype(np.int64)
train_test['year']=train_test['year'].astype(np.int64)

In [114]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,year,month,...,city,state,type,cluster,date_str,national_holiday,national_event,national_workday,local_holiday,regional_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4,0,2017,8,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0


To confirm one last time that the only missing values are the expected sales values from the original test dataframe. 

In [115]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

sales    28512
dtype: int64

To double check there are still a valid number of holidays listed. 

In [116]:
train_test.national_holiday.value_counts() 

0    2787048
1     242352
Name: national_holiday, dtype: int64

In [117]:
train_test.regional_holiday.value_counts() 

0    3028377
1       1023
Name: regional_holiday, dtype: int64

In [118]:
train_test.local_holiday.value_counts() 

0    3017949
1      11451
Name: local_holiday, dtype: int64

In [125]:
train_test_encoded = train_test

In [127]:
from sklearn import preprocessing
categories = ['family','city','state','type']
for i in categories:
    encoder = preprocessing.LabelEncoder()
    train_test_encoded[i] = encoder.fit_transform(train_test_encoded[i])

In [128]:
train_test_encoded

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,year,month,...,city,state,type,cluster,date_str,national_holiday,national_event,national_workday,local_holiday,regional_holiday
0,0,2013-01-01,1,0,0.0,0,2,0,2013,1,...,18,12,3,13,2013-01-01,1,0,0,0,0
1,1,2013-01-01,1,1,0.0,0,2,0,2013,1,...,18,12,3,13,2013-01-01,1,0,0,0,0
2,2,2013-01-01,1,2,0.0,0,2,0,2013,1,...,18,12,3,13,2013-01-01,1,0,0,0,0
3,3,2013-01-01,1,3,0.0,0,2,0,2013,1,...,18,12,3,13,2013-01-01,1,0,0,0,0
4,4,2013-01-01,1,4,0.0,0,2,0,2013,1,...,18,12,3,13,2013-01-01,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,28,NaN,1,4,0,2017,8,...,18,12,1,6,2017-08-31,0,0,0,0,0
3029396,3029396,2017-08-31,9,29,NaN,0,4,0,2017,8,...,18,12,1,6,2017-08-31,0,0,0,0,0
3029397,3029397,2017-08-31,9,30,NaN,1,4,0,2017,8,...,18,12,1,6,2017-08-31,0,0,0,0,0
3029398,3029398,2017-08-31,9,31,NaN,9,4,0,2017,8,...,18,12,1,6,2017-08-31,0,0,0,0,0


To split the train and test dataframes back out. 

In [119]:
train_features = train_test[train_test['sales'].notnull()].copy()
test_features = train_test[train_test['sales'].isnull()].drop(['sales'],axis=1)

In [120]:
train_features

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,year,month,...,city,state,type,cluster,date_str,national_holiday,national_event,national_workday,local_holiday,regional_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.000,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.000,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.000,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.000,0,2,0,2013,1,...,Quito,Pichincha,D,13,2013-01-01,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3028867,3000883,2017-08-15,9,POULTRY,438.133,0,2,0,2017,8,...,Quito,Pichincha,B,6,2017-08-15,0,0,0,0,0
3028868,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2,0,2017,8,...,Quito,Pichincha,B,6,2017-08-15,0,0,0,0,0
3028869,3000885,2017-08-15,9,PRODUCE,2419.729,148,2,0,2017,8,...,Quito,Pichincha,B,6,2017-08-15,0,0,0,0,0
3028870,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2,0,2017,8,...,Quito,Pichincha,B,6,2017-08-15,0,0,0,0,0


In [121]:
test_features

,id,date,store_nbr,family,onpromotion,day_of_week,weekend,year,month,day_of_month,...,city,state,type,cluster,date_str,national_holiday,national_event,national_workday,local_holiday,regional_holiday
55572,3000888,2017-08-16,1,AUTOMOTIVE,0,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
55573,3000889,2017-08-16,1,BABY CARE,0,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
55574,3000890,2017-08-16,1,BEAUTY,2,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
55575,3000891,2017-08-16,1,BEVERAGES,20,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
55576,3000892,2017-08-16,1,BOOKS,0,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,1,4,0,2017,8,31,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029396,3029396,2017-08-31,9,PREPARED FOODS,0,4,0,2017,8,31,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029397,3029397,2017-08-31,9,PRODUCE,1,4,0,2017,8,31,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,4,0,2017,8,31,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0


In [123]:
train_features['city'].unique()

array(['Quito', 'Cayambe', 'Latacunga', 'Riobamba', 'Ibarra',
       'Santo Domingo', 'Guaranda', 'Puyo', 'Ambato', 'Guayaquil',
       'Salinas', 'Daule', 'Babahoyo', 'Quevedo', 'Playas', 'Libertad',
       'Cuenca', 'Loja', 'Machala', 'Esmeraldas', 'Manta', 'El Carmen'],
      dtype=object)

In [124]:
test_features['city'].unique()

array(['Quito', 'Cayambe', 'Latacunga', 'Riobamba', 'Ibarra',
       'Santo Domingo', 'Guaranda', 'Puyo', 'Ambato', 'Guayaquil',
       'Salinas', 'Daule', 'Babahoyo', 'Quevedo', 'Playas', 'Libertad',
       'Cuenca', 'Loja', 'Machala', 'Esmeraldas', 'Manta', 'El Carmen'],
      dtype=object)

To save the train_features and test_features as new csv files that can be used in the next stage of the project. 

In [122]:
train_features.to_csv('train_features.csv', index=False)
test_features.to_csv('test_features.csv', index=False)

In [129]:
train_encoded = train_test_encoded[train_test_encoded['sales'].notnull()].copy()
test_encoded = train_test_encoded[train_test_encoded['sales'].isnull()].drop(['sales'],axis=1)

In [130]:
train_encoded.to_csv('train_encoded.csv', index=False)
test_encoded.to_csv('test_encoded.csv', index=False)